In [133]:
from scipy.spatial import distance
from itertools import combinations_with_replacement
import numpy as np
import random

from joblib import Parallel, delayed 
import multiprocessing 

initial_chord = [64, 67, 71, 74] #Defines by midi number the notes of the chord given by the user

upper_time_signature = 4 #Indicates how many beats are grouped together in a measure(bar), value given by the user
lower_time_signature = 4 #Indicates the note value that represents one beat, value given by the user 

#Silulating a melody created by the algorithm, p = pitch and r = rhythm
#p = [60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77]
#r = [0.5, 0.5, 0.5, 0.5, 1.5, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0]

#L = sum(r)
#M = int(L / upper_time_signature) #M defines the number of measures in the melody

if lower_time_signature == 2:
    beat = 2
elif lower_time_signature == 4:
    beat = 1
elif lower_time_signature == 8:
    beat = 0.5 

#D = 10

#print(N, L, M)#teste

In [134]:
!pip install -U pyrecorder
from pyrecorder.recorders.file import File
from pyrecorder.video import Video

Requirement already up-to-date: pyrecorder in c:\users\rafael\anaconda3\lib\site-packages (0.1.8)


In [135]:
from pymoo.model.problem import Problem
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.optimize import minimize
from pymoo.model.repair import Repair

class MusicGeneration(Problem):    
    #Here start the equation to define the fitness function on stability
    #CT calculates a fraction of CHORD TONES
    #Eq.(3)
    def CT(p, r, N):
        list_CT = [] 
        for i in range(N):
            num = MusicGeneration.C(p[i]) * r[i]
            den = sum(r)
            list_CT.append(num/den)
        return sum(list_CT)

    #Cn returns 1 when the note is a chord note, else 0
    #Eq.(4)
    def C(p):
        return 1 if p in initial_chord else 0

    #SM calculates te amount of step motion(midi value less than 2) in p
    #Asigns a higher score in the range of [0.0, 1.0] to the melody that contains more step motions. 
    #Eq.(5)
    def SM(p, N):
        list_SM = []
        for i in range(N-1):
            num = MusicGeneration.U(2 - abs(p[i] - p[i+1]))
            den = N-1
            list_SM.append(num/den)
        return sum(list_SM)

    #Eq.(6)
    def U(x): 
        return 1 if x>=0 else 0

    #DL evaluates a fraction if three consecutive notes in descending lines of the pitch sequence p
    #Returns 1 only if the three successive notes are descending    
    #Eq.(7)
    def DL(p, N):
        list_DL = []
        for i in range(N-2):
            num = MusicGeneration.U(p[i] - p[i+1] - 1) * MusicGeneration.U(p[i+1] - p[i+2] - 1)
            den = N-2
            list_DL.append(num/den)
        return sum(list_DL)

    #Here start the equation to define the fitness function on tension

    #NT computes a fraction of non-chord tone(notes that is not in initial_chord) of the melody
    #Eq.(9)
    def NT(p, r, N):
        return 1.0 - MusicGeneration.CT(p, r, N)

    #LM measures a ratio of skip motion(notes that are connected by a midi value higher than 2)
    #Returns 1 when two consecutive notes are connected by a skip motion
    #Thus, LM(p) returns a higher value when the pitch vector p is comprised of more skip motions
    #Eq.(10)
    def LM(p, N):
        list_LM = []
        for i in range(N-1):
            num = MusicGeneration.U(abs(p[i] - p[i+1]) - 2)
            den = N-1
            list_LM.append(num/den)
        return sum(list_LM)

    #AL evaluates a fraction of three successive notes in ascending lines (in contrast with DL(p, N))
    #Eq.(11)
    def AL(p, N):
        list_AL = []
        for i in range(N-2):
            num = MusicGeneration.U(p[i+2] - p[i+1] - 1) * MusicGeneration.U(p[i+1] - p[i] - 1)
            den = N-2
            list_AL.append(num/den)
        return sum(list_AL)

    #Final equation to calculate the penalty

    #Eq.(12)
    def P(p, r, full_pitch_list, full_melody_list, N, S):
        M = int(sum(r) / upper_time_signature)
        return MusicGeneration.MNC(p, N, S), MusicGeneration.AN(p, S), MusicGeneration.ON(full_pitch_list, full_melody_list, M), MusicGeneration.BL(p, N)

    ###Defining the rules on the plausible pleasant non-chord tones
    ##Rule 1: Non-chord tone on the strong beat should move to a chord tone by a step motion

    #UT measures the number of unresolved non-chord tones on the strong beats
    #S is a set of indices of the strong beats 
    #Eq.(13)
    def UT(p, S):
        list_UT = []
        for i in S:
            num = MusicGeneration.Nn(p[i])
            if i != len(p) - 1:
                num *= (MusicGeneration.Nn(p[i+1]) + MusicGeneration.U(abs(p[i] - p[i+1])) - 2)
            list_UT.append(num)
        return sum(list_UT)

    #Eq.(14)
    def Nn(p): 
        return 1 if p not in initial_chord else 0

    ##Rule 2: All non-chord tones should be connected with at least one adjacent chord tone by a step motion

    #Eq.(15)
    def NCC(p, N):
        list_NCC = []
        for i in range(N-1): #correction in the equation
            num = (MusicGeneration.Nn(p[i])) * (MusicGeneration.Nn(p[i-1]) +
                                                MusicGeneration.U(abs(p[i-1] - p[i]) - 2)) * (
                MusicGeneration.Nn(p[i+1]) + MusicGeneration.U(abs(p[i] - p[i+1]) - 2))
            list_NCC.append(num)
        return sum(list_NCC)

    ##Rule 3: Skip motion on the non-chord tones should not be over the third interval

    #NCL Captures the number of intervals which are greater than the third with an adjacent non-chord tone
    #Eq.(16)
    def NCL(p, N):
        list_NCL = []
        for i in range(N-1): #correction in the equation
            num = MusicGeneration.Nn(p[i]) * (MusicGeneration.U(4 - abs(p[i-1] - p[i])) +
                                              MusicGeneration.U(4 - abs(p[i] - p[i+1])))
            list_NCL.append(num)
        return sum(list_NCL)

    #Here start the defining the penalty function

    #Minimizing MNC(p) can resolve a misused non-chord tones in the melody
    #Eq.(17)
    def MNC(p, N, S):
        return MusicGeneration.UT(p, S) + MusicGeneration.NCC(p, N) + MusicGeneration.NCL(p, N)

    #AN investigate the number o avoid notes over the strong beats
    #If both Nn(p[i]) and C(p[i] - 1) return 1, p[i] becomes a avoid note
    #Eq.(18)
    def AN(p, S):
        list_AN = []
        for i in S:
            num = MusicGeneration.Nn(p[i]) * MusicGeneration.C(p[i] - 1)
            list_AN.append(num)
        return sum(list_AN)

    #ON counts the number of measures(i.e., bar) that contains more than 50% of non-chord tones
    #Eq.(19)        
    def ON2(p, r):
        list_ON2 = []
        for j in range(len(p)):
            num = r[j] * MusicGeneration.C(p[j])
            list_ON2.append(num/sum(r))
        return sum(list_ON2)

    def ON(full_pitch_list, full_melody_list, M):
        list_ON = []
        for i in range(M):
            num = MusicGeneration.U(0.5 - MusicGeneration.ON2(full_pitch_list[i], full_melody_list[i]))
            list_ON.append(num)
        return sum(list_ON)

    #BL(p) computes the number of over-leap intervals
    #Eq.(20)
    def BL(p, N):
        list_BL = []
        for i in range(N-1):
            num = MusicGeneration.U(16 - abs(p[i] - p[i+1]))
            list_BL.append(num)
        return sum(list_BL)

    #This function defines and separates the measures in the melody
    def divide(rhythm_list, pitch_list, target):
        # vamos começar a contagem da soma em zero
        count = 0
        count_beat = 0

        # aqui está a nossa lista com as "fatias" que queremos
        full_melody_list = []
        full_pitch_list = []
        S_index = []

        # aqui fica uma measure
        measure = []
        measure_pitch = []
        
        for i in range(len(rhythm_list)):
            item = rhythm_list[i]
            count += item # aqui somamos o nosso contador
            measure.append(item) # adicionamos o item atual na lista da measure
            measure_pitch.append(pitch_list[i])
        
            # quando o contador bate o target "fecharemos" o measure atual
            # e prepararemos um novo measure e resetaremos o contador
            if count >= target:
                full_melody_list.append(measure)
                full_pitch_list.append(measure_pitch)
                measure = []
                measure_pitch = []
                count = 0
        
        # adicionando algum resto, caso exista
        if len(measure) > 0:
            full_melody_list.append(measure)
            full_pitch_list.append(measure_pitch)
        
        return full_melody_list, full_pitch_list

    def get_s_index(full_melody_list, full_pitch_list):
        number_measures = len(full_melody_list)

        # processando cada measure individualmente
        current_lower_index = 0
        S_index = np.array([])

        for i in range(number_measures):
            current_pitch = np.array(full_pitch_list[i])
            current_melody = np.array(full_melody_list[i])

            # 1a regra: soma dos primeiros rhythms não-negativos que alcancem o valor do beat
            # pegando somente o índice dos pitches positivos
            positive_pitch_indexes = np.argwhere(np.array(full_pitch_list[i]) > 0).flatten()
            
            # somando os rhythms desta measure até dar o beat
            soma = np.cumsum(current_melody[positive_pitch_indexes])
            threshold = np.argwhere(soma >= beat)[0][0]
            S_index = np.append(np.array(range(threshold+1)) + current_lower_index, S_index)
            
            # 2a regra: considerar upper time signature
            beats, _ = MusicGeneration.divide(full_melody_list[i], full_pitch_list[i], beat)
            S_index = np.append(np.array(range(len(beats[0]))) + current_lower_index, S_index)
            
            if upper_time_signature == 4 or upper_time_signature == 6:
                previous_beats = len(np.array(beats[:int(upper_time_signature/2)]).flatten())
                
                # existem alguns casos onde o len_current_beat não foi encontrado
                try:
                    len_current_beat = len(np.array(beats[int(upper_time_signature/2)]).flatten())
                    S_index = np.append(np.array(range(previous_beats, previous_beats + len_current_beat)) +
                          current_lower_index, S_index)
                except:
                    S_index = np.append(np.array(range(previous_beats, previous_beats)) + current_lower_index, S_index)
                    
            current_lower_index += len(current_pitch)

        return np.unique(S_index).astype(int)

    #fitness_1 calculates which is the higher stability value and assigns it with alpha, beta, gamma
    #A higher fitness value presents a better melody in terms of stability
    #Eq.(2)
    def fitness_1(self, p_list, r_list, N):
        fitness_results = []
        
        for i in range(len(p_list)):
            p = p_list[i]
            r = r_list[i]
            
            '''
            list_fitness_1 = []
            list_fitness_1.append(MusicGeneration.CT(p, r, N))
            list_fitness_1.append(MusicGeneration.SM(p, N))
            list_fitness_1.append(MusicGeneration.DL(p, N))
            fitness_1_sorted = sorted(list_fitness_1, reverse = True)
            
            more_stability = fitness_1_sorted[0]
            midle_stability = fitness_1_sorted[1]
            less_stability = fitness_1_sorted[2]
            
            full_melody_list, full_pitch_list = MusicGeneration.divide(r, p, upper_time_signature)
            S = MusicGeneration.get_s_index(full_melody_list, full_pitch_list)
            
            fitness_results.append(-((self.alpha*more_stability) + 
                                    (self.beta*midle_stability) + 
                                    (self.gamma*less_stability)) - (self.delta*MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S)))
            '''
            '''
            full_melody_list, full_pitch_list = MusicGeneration.divide(r, p, upper_time_signature)
            S = MusicGeneration.get_s_index(full_melody_list, full_pitch_list)
            
            fitness_results.append(((self.alpha*MusicGeneration.CT(p, r, N)) + 
                                    (self.beta*MusicGeneration.SM(p, N)) + 
                                    (self.gamma*MusicGeneration.DL(p, N))) - (
                self.delta*MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S)))
            '''
            fitness_results.append((self.alpha*MusicGeneration.CT(p, r, N)) + 
                                   (self.beta*MusicGeneration.SM(p, N)) + 
                                   (self.gamma*MusicGeneration.DL(p, N)))
            
        return fitness_results

    #fitness_2 calculates which is the higher tension value and assigns it with alpha, beta, gamma
    #A higher fitness value presents more instability on the melody
    #Eq.(8)
    def fitness_2(self, p_list, r_list, N):
        fitness_results = []
        
        for i in range(len(p_list)):
            p = p_list[i]
            r = r_list[i]
            '''
            list_fitness_2 = []
            list_fitness_2.append(MusicGeneration.NT(p, r, N))
            list_fitness_2.append(MusicGeneration.LM(p, N))
            list_fitness_2.append(MusicGeneration.AL(p, N))
            fitness_2_sorted = sorted(list_fitness_2, reverse = True)
            
            more_tension = fitness_2_sorted[0]
            midle_tension = fitness_2_sorted[1]
            less_tension = fitness_2_sorted[2]
            
            full_melody_list, full_pitch_list = MusicGeneration.divide(r, p, upper_time_signature)
            S = MusicGeneration.get_s_index(full_melody_list, full_pitch_list)
            
            fitness_results.append(-((self.alpha*more_tension) +
                                    (self.beta*midle_tension) + 
                                    (self.gamma*less_tension)) - (self.delta*MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S)))
            '''
            '''
            full_melody_list, full_pitch_list = MusicGeneration.divide(r, p, upper_time_signature)
            S = MusicGeneration.get_s_index(full_melody_list, full_pitch_list)
            
            fitness_results.append(((self.alpha*MusicGeneration.NT(p, r, N)) +
                                    (self.beta*MusicGeneration.LM(p, N)) + 
                                    (self.gamma*MusicGeneration.AL(p, N))) - (
                self.delta*MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S)))
            '''
            fitness_results.append((self.alpha*MusicGeneration.NT(p, r, N)) +
                                   (self.beta*MusicGeneration.LM(p, N)) + 
                                   (self.gamma*MusicGeneration.AL(p, N)))
            
        return fitness_results
    
    def penalty(self, p_list, r_list, N):
        fitness_results = []
        
        for i in range(len(p_list)):
            p = p_list[i]
            r = r_list[i]
            '''
            list_fitness_2 = []
            list_fitness_2.append(MusicGeneration.NT(p, r, N))
            list_fitness_2.append(MusicGeneration.LM(p, N))
            list_fitness_2.append(MusicGeneration.AL(p, N))
            fitness_2_sorted = sorted(list_fitness_2, reverse = True)
            
            more_tension = fitness_2_sorted[0]
            midle_tension = fitness_2_sorted[1]
            less_tension = fitness_2_sorted[2]
            
            full_melody_list, full_pitch_list = MusicGeneration.divide(r, p, upper_time_signature)
            S = MusicGeneration.get_s_index(full_melody_list, full_pitch_list)
            
            fitness_results.append(-((self.alpha*more_tension) +
                                    (self.beta*midle_tension) + 
                                    (self.gamma*less_tension)) - (self.delta*MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S)))
            '''
            full_melody_list, full_pitch_list = MusicGeneration.divide(r, p, upper_time_signature)
            S = MusicGeneration.get_s_index(full_melody_list, full_pitch_list)
            
            #fitness_results.append(self.delta*MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S))
            fitness_results.append(MusicGeneration.P(p, r, full_pitch_list, full_melody_list, N, S))
            
        return fitness_results
        
    
    # pymoo functions, https://www.pymoo.org/problems/index.html#User-defined-Problems
    def __init__(self, n_var, xl, xu, accepted_penalty, alpha=3.0, beta=2.0, gamma=1.0, delta=100.0):
        # defining alpha, beta, gamma and delta to be used in the optimization process
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.accepted_penalty = accepted_penalty
        
        super().__init__(n_var=n_var, # number of notes
                         n_obj=3, # number of objectives (fitness_1 and fitness_2)
                         n_constr=4, # number of constraints (0)
                         xl=xl, # lower bounds (the minimum value to each pitch and rhythm)
                         xu=xu) # upper bounds (the maximum value to each pitch and rhythm)

    def _evaluate(self, x, out, *args, **kwargs):
        N = int(self.n_var/2)
        
        p = x[:, :N] # p is the first part of the solution
        r = x[:, N:] # r is the second part of the solution
        
        out["F"] = np.column_stack([self.fitness_1(p, r, N), self.fitness_2(p, r, N)])*-1
        out["G"] = np.column_stack([self.penalty(p, r, N)]) - self.accepted_penalty
        
class RepairMusic(Repair):
    #This function defines and separates the measures in the melody
    def divide(self, rhythm_list, target):
        standard_rhythms = [4.0, 3.0, 2.0, 1.5, 1.0, 0.75, 0.5, 0.375, 0.25]
        
        # vamos começar a contagem da soma em zero
        count = 0
        count_beat = 0

        # aqui está a nossa lista com as "fatias" que queremos
        full_melody_list = []
        S_index = []

        # aqui fica uma measure
        measure = []
        for i in range(len(rhythm_list)):
            item = rhythm_list[i]
            count += item # aqui somamos o nosso contador
            measure.append(item) # adicionamos o item atual na lista da measure
        
            # quando o contador bate o target "fecharemos" o measure atual
            # e prepararemos um novo measure e resetaremos o contador
            if count >= target:
                full_melody_list.append(measure)
                measure = []
                measure_pitch = []
                count = 0
        
        # adicionando algum resto, caso exista
        if len(measure) > 0:
            full_melody_list.append(measure)
        
        # agora que fechamos as divisões ajustaremos internamente para que a soma seja sempre o target
        full_melody_list_fixed = []
        for melody in full_melody_list:
            # getting all the combinations respecting the number of elements
            combinations = np.array(list(combinations_with_replacement(standard_rhythms, len(melody))))

            # filtering only the combinations which sum is equal to the target
            combinations = combinations[np.where(np.sum(combinations, axis=1)==target)]

            # attempting to get the most similar melodies to the current list
            equal_elements = np.sum(combinations==melody, axis=1)

            # calculating the most similar combinations through cosine distance
            distances = np.array([distance.cosine(melody, combination) for combination in combinations])

            # the closer to 1, the better
            full_melody_list_fixed.extend(combinations[np.argmax(distances)])
            
        return full_melody_list_fixed
    
    def _do(self, problem, pop, **kwargs):
        # each row one individual
        Z = pop.get("X")
        N = int(Z.shape[1]/2)
        
        # now repair each individual i
        num_cores = multiprocessing.cpu_count()
        Z[:, :N] = Z[:, :N].astype(int)
        Z[:, N:] = Parallel(n_jobs=num_cores)(delayed(self.divide)(Z[i, N:], upper_time_signature) for i in range(len(Z)))
        
        # set the design variables for the population
        pop.set("X", Z)
        return pop
        
n_notes = 14

lower_bounds = [1]*n_notes + [0.50]*n_notes #52 88 0.50 testando
upper_bounds = [127]*n_notes + [4]*n_notes
        
# defining the problem and the algorithm to be used
problem = MusicGeneration(n_var=n_notes*2, xl=lower_bounds, xu=upper_bounds,
                          alpha=3.0, beta=2.0, gamma=1.0, delta=100.0, accepted_penalty=3)

algorithm = NSGA2(pop_size=25, repair=RepairMusic())

# starting the optimization process
res = minimize(problem,
               algorithm,
               ('n_gen', 50),
               seed=0,
               save_history=True,
               verbose=True)

<ipython-input-135-a3962cc3fead>:233: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  previous_beats = len(np.array(beats[:int(upper_time_signature/2)]).flatten())


n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |      25 |  3.60000E+01 |  5.01200E+01 |       1 |            - |            -
    2 |      50 |  3.40000E+01 |  4.48800E+01 |       1 |  0.174679487 |        ideal
    3 |      75 |  3.40000E+01 |  4.03200E+01 |       1 |  0.00000E+00 |            f
    4 |     100 |  3.20000E+01 |  3.74800E+01 |       1 |  0.241346154 |        ideal
    5 |     125 |  3.20000E+01 |  3.46800E+01 |       1 |  0.00000E+00 |            f
    6 |     150 |  2.60000E+01 |  3.31200E+01 |       1 |  0.343108974 |        ideal
    7 |     175 |  2.60000E+01 |  3.22800E+01 |       1 |  0.00000E+00 |            f
    8 |     200 |  2.50000E+01 |  3.07600E+01 |       1 |  0.375000000 |        ideal
    9 |     225 |  2.00000E+01 |  2.82000E+01 |       1 |  0.143429487 |        ideal
   10 |     250 |  2.00000E+01 |  2.49200E+01 |       1 |  0.00000E+00 |            f
   11 |     275 |  1.30000E+01 |  2.37600E+01 |       

In [136]:
from pymoo.visualization.scatter import Scatter

Scatter().add(res.F).show()

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [ ]:
# use the video writer as a resource
with Video(File("nsga.mp4")) as vid:

    # for each algorithm object in the history
    for entry in res.history:
        sc = Scatter(title=("Gen %s" % entry.n_gen))
        sc.add(entry.pop.get("F"))
        sc.add(entry.problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
        sc.do()

        # finally record the current visualization to the video
        vid.record()

## Gerando as melodias em MIDI

In [ ]:
import random
import pretty_midi
from IPython.display import Audio

#### Gerando para o caso com máxima tensão, depois com máxima estabilidade, depois um equilibrado

In [ ]:
max_estabilidade = np.argmin(res.F[:, 0])
max_tensao = np.argmin(res.F[:, 1])

In [ ]:
def generate_music(index):
    # Create a PrettyMIDI object
    music = pretty_midi.PrettyMIDI()

    # Create an Instrument instance for an instrument
    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    
    melody = res.X[index]
    length = int(len(melody)/2)
    p = melody[:length].astype(int)
    r = melody[length:]
    display(f'p:{p},r:{r}')
    
    start = 0
    for item in range(len(p)):
        end = start + r[item]
        note = pretty_midi.Note(pitch=p[item], start=start, end=end, velocity=100)
        piano.notes.append(note)
        
        # preparing for the next start
        start += r[item]
        
    # Add the cello instrument to the PrettyMIDI object
    music.instruments.append(piano)

    # Write out the MIDI data
    music.write(f'music_{index}.mid')
    return music

In [ ]:
!pip install pyFluidSynth
#import fluidsynth
#import pyFluidSynth

display('Máxima estabilidade: ')
display(Audio(generate_music(max_estabilidade).fluidsynth(fs=16000), rate=16000, autoplay=False))

display('Máxima tensão: ')
display(Audio(generate_music(max_tensao).fluidsynth(fs=16000), rate=16000, autoplay=False))

In [ ]:
for index in range(len(res.F)):
    display(f'Índice {index} ({res.F[index, :]})')
    display(Audio(generate_music(index).fluidsynth(fs=16000), rate=16000, autoplay=False))